In [3]:
from analysis import actions, findBillNum, date_day, collectConnections, getHistory, getReferencedBills
from dateutil.parser import parse
import pandas as pd
import json
import re
from matplotlib import pyplot as plt
import copy


bills = set([x.id for x in actions])


actions_by_bill = [getHistory(x) for x in bills]






In [7]:

committees = set()
for actions in actions_by_bill:
  print_all = False
  for a in actions: 
    # c = re.compile(".*(committee on.*)", re.IGNORECASE).findall(a.action)
    # if len(c) != 0: 
    #   c.append(a.branch)
    #   committees.add(tuple(c))
    d = re.compile("(.*[house|senate].*transportation.*)", re.IGNORECASE).findall(a.action)
    if len(d) != 0:
      print_all = True
      # d.append(a.branch)
      # committees.add(tuple(d))
  if print_all == True: 
    print([x.action for x in actions])

['Referred to the committee on Transportation', 'Senate concurred', 'Hearing scheduled for 01/10/2022 from 02:00 PM-05:00 PM in Virtual Hearing', 'Bill reported favorably by committee and referred to the committee on House Ways and Means']
['Referred to the committee on Transportation', 'Senate concurred', 'Hearing scheduled for 01/19/2022 from 03:00 PM-06:00 PM in Virtual Hearing']
['Referred to the committee on Public Service', 'House concurred', 'Discharged to the committee on Transportation', 'House concurred', 'Hearing scheduled for 01/10/2022 from 02:00 PM-05:00 PM in Virtual Hearing', 'Reporting date extended to Friday March 4, 2022, pending concurrence', 'Senate concurred', 'Accompanied a new draft, see H4619']
['Referred to the committee on Transportation', 'Senate concurred', 'Hearing scheduled for 06/02/2021 from 01:00 PM-03:00 PM in Virtual Hearing', 'Accompanied a new draft, see H3942']
['Referred to the committee on Transportation', 'House concurred', 'Hearing scheduled f

In [5]:

for s in sorted(committees):
  print(f"[{s[1]}]: {s[0]}")
    # committee.append(re.compile("(house|senate|joint).*committee(.*)", re.IGNORECASE).findall(a.action))
    # if re.compile('hearing', re.IGNORECASE).search(a.action) is not None:
    #   if a.branch == "Joint":
    #     print(a.id, a.branch, a.action)
    #     print(committee)

In [ ]:
import requests


# f = open ("../data/committees-list.json")
# committee_list = json.loads(f.read())
# f.close()



# def get_url(committee_item):
#   return committee_item['Details']


# for i in committee_list: 
#   url = get_url(i)
#   info = requests.get(url, verify=False)
  
#   infojson = info.json()
#   print(infojson['FullName'])
  
  

In [ ]:

# committee_details = []

# g = open("../data/committees-detail.json", "w")
# g.write("[")

# for i in committee_list:
#   url = get_url(i)
#   info = requests.get(url, verify=False)
  
#   g.write(info.text)
#   g.write(", ")
# g.write("]")
# g.close()

In [64]:
f = open("../data/committees-detail.json")


committees_list = json.loads(f.read())


In [95]:



df_committees = pd.DataFrame(committees_list)

df_committees.describe()

print(df_committees.columns)
df_committees['count_docs'] = df_committees['DocumentsBeforeCommittee'].apply(lambda x : len(x))

display(df_committees[['count_docs', 'FullName']])



Index(['CommitteeCode', 'FullName', 'ShortName', 'Description', 'Branch',
       'GeneralCourtNumber', 'SenateChairperson', 'HouseChairperson',
       'DocumentsBeforeCommittee', 'ReportedOutDocuments', 'Hearings'],
      dtype='object')


,count_docs,FullName
0,97,Senate Committee on Bills in the Third Reading
1,214,Senate Committee on Rules
2,0,Senate Committee on Global Warming and Climate...
3,0,Senate Committee on Post Audit and Oversight
4,0,Senate Committee on Steering and Policy
5,576,Senate Committee on Ways and Means
6,0,Senate Committee on Ethics
7,0,Senate Committee on Intergovernmental Affairs
8,0,Senate Committee on Redistricting
9,0,Senate Committee on Personnel and Administration


In [2]:
df = pd.read_csv('../data/all-history-actions.csv')

groups = df.groupby(by="id")

bills = [i for i, g in groups]

In [3]:
def define_bill_set(target_size):
  visited = set()
  graphable_bills = set()
  
  for b in bills: 
    if len(graphable_bills) > 1200:
      break 
    if b not in visited and b not in graphable_bills: 
      cloud = collectConnections(b) 
      if len(cloud) == target_size:
        graphable_bills.add(b)
      visited.update(cloud)
  
  return graphable_bills


graphable_bills = define_bill_set(6)


In [4]:


def connectionGraph(bill):
  cloud = list(collectConnections(bill))
  cloud.append(bill)

  nodes = {}
  
  for bill in cloud:
    hist = getHistory(bill)
    connections = getReferencedBills(hist)
    connect_dates = []
    
    for h in hist: 
      pair = findBillNum.findall(h.action)
      for p in pair: 
        p_hist = getHistory(p)
        for ph in p_hist: 
          if ph.action.count(bill) > 0: 
             connect_dates.append({ 
                "x1": h.date, 
                "x2": ph.date or h.date, 
                "y1": h.id, 
                "y2": ph.id, 
                "action1": h.action, 
                "action2": ph.action
              })
        
    
      
      nodes[bill] = {'id': bill, 
              'connections': connections, 
              'connect_dates': connect_dates,
              'start': hist[0].date,
              'end': hist[-1].date,
              'history': hist
    }
    

  return nodes

# return: list of dicts that list id, all bills it's connected to, the array of actions
test_graph = connectionGraph('H3456')



In [5]:


def search_graph(nodes): 
  nodes = copy.deepcopy(nodes)
  
  starting_node = max(nodes.values(), key=lambda x: x['end'])
  starting_node['depth'] = 0
  starting_node['parent'] = (starting_node['id'], starting_node['start'])
  
  queue = [starting_node] 
  visited = []
  result = []
  
  while len(queue) > 0: 
    n = queue.pop()
    
    sorted_nodes = sorted([nodes[bill_num] for bill_num in n['connections']], key=lambda x: x['id'])
    
    for node in sorted_nodes: 
      if node['id'] not in visited:
        node['depth'] = n['depth'] + 1
        node['parent'] = (n['id'], n['start'], n['end'])
        
        queue.append(node)
    visited.append(n['id'])
    result.append(n)
  
  
  
  return sorted(result, key=lambda x: x['id'])
  
  
result = search_graph(test_graph)



In [6]:

def graph_cloud_by_dates(bill):
  cloud_nodes = connectionGraph(bill)
  graphable = search_graph(cloud_nodes)

  plt.figure(figsize=(20,5))
  
  
  for n in sorted(graphable, key=lambda x: x['id']):
    start = n['start']
    end = n['end']
    
    points = [n['id'], start, [n['id'], end]]    

    x = [parse(start), parse(end)]
    y = [n['id'], n['id']]
    
    plt.plot(x , y , '>-', linewidth=2, label=n["id"])
  
    
  plt.legend()
  plt.show()


# graph_cloud_by_dates("S3030")



In [7]:


def d3_data(bill):
  cloud_nodes = connectionGraph(bill)
  graphable = search_graph(cloud_nodes)

  node_index = ['index', 'id', 'connections', 'connect_dates', 'start', 'end', 'history', 'depth', 'parent']
  action_index = ['index', 'over_index', 'id', 'action', 'branch', 'date']
  
  
  df_bills = pd.DataFrame(graphable)
  
  
  bill = []
  start_date = []
  
  data = []
  
  
  for g in df_bills.itertuples(): 
    g_series = pd.Series(g, index=node_index)
    
    connected_dates = g_series.connect_dates
    history_dates = pd.Series(g.history)
  
    d = {'id': g_series.id, 
        'start': g_series.start, 
        'end': g_series.end, 
        'connect_dates': connected_dates,
        'depth': g_series.depth, 
        'parent': g_series.parent,
        'history': [{"date": g.date, "action": g.action} for g in history_dates]
    }
    data.append(d)
  
  return json.dumps(data)

    
    
dump = json.dumps(json.loads(d3_data("H1")))



In [9]:

print(dump)

[{"id": "H1", "start": "2021-02-25T16:46:19.933Z", "end": "2021-04-15T04:00:00.000Z", "connect_dates": [{"x1": "2021-04-15T04:00:00.000Z", "x2": "2021-04-15T04:00:00.000Z", "y1": "H1", "y2": "H4000", "action1": "Reported, in part, by H4000", "action2": "Reported on a part of H1"}], "depth": 5, "parent": ["H4000", "2021-04-15T04:00:00.000Z", "2021-05-03T04:00:00.000Z"], "history": [{"date": "2021-02-25T16:46:19.933Z", "action": "Referred to the committee on House Ways and Means"}, {"date": "2021-04-15T04:00:00.000Z", "action": "Reported, in part, by H4000"}]}, {"id": "H4000", "start": "2021-04-15T04:00:00.000Z", "end": "2021-05-03T04:00:00.000Z", "connect_dates": [{"x1": "2021-04-15T04:00:00.000Z", "x2": "2021-04-15T04:00:00.000Z", "y1": "H4000", "y2": "H1", "action1": "Reported on a part of H1", "action2": "Reported, in part, by H4000"}, {"x1": "2021-05-03T04:00:00.000Z", "x2": "2021-05-03T04:00:00.000Z", "y1": "H4000", "y2": "H4001", "action1": "Published as amended, see H4001", "acti

In [8]:
batch_bills = []

for bill in bills[10:20]:
  batch_bills.extend(json.loads(d3_data(bill)))


display(json.dumps(batch_bills))



'[{"id": "H1007", "start": "2021-03-29T14:50:09.363Z", "end": "2022-05-16T04:00:00.000Z", "connect_dates": [{"x1": "2022-05-16T04:00:00.000Z", "x2": "2022-05-16T04:00:00.000Z", "y1": "H1007", "y2": "S2880", "action1": "Accompanied a new draft, see S2880", "action2": "New draft of S495, S502, H860, H967 and H1007"}], "depth": 1, "parent": ["S2880", "2022-05-16T04:00:00.000Z", "2022-05-16T15:07:18.147Z"], "history": [{"date": "2021-03-29T14:50:09.363Z", "action": "Referred to the committee on Environment, Natural Resources and Agriculture"}, {"date": "2021-03-30T03:59:59.000Z", "action": "Senate concurred"}, {"date": "2021-05-11T16:50:31.467Z", "action": "Hearing scheduled for 05/18/2021 from 01:00 PM-05:00 PM in Virtual Hearing"}, {"date": "2022-02-07T16:30:10.047Z", "action": "Reporting date extended to Thursday March 31, 2022, pending concurrence"}, {"date": "2022-03-03T16:11:18.947Z", "action": "Senate concurred"}, {"date": "2022-04-04T15:29:49.097Z", "action": "Reporting date extend